# Preprocessing data stoplocatie detectie
In dit script voeren we data preprocessing uit om straks een model te kunnen bouwen die automatisch stoplocaties detecteert met GPS data van schepen. 

In de ruwe data hebben we informatie over de mmsi van het schip, schipnaam, longitude, latitude, tijd, snelheid, orientatie,  heading, navigatiestatus en shiptype. 

Met informatie over longitude (x), latitude (y) en tijd (t) creëeren we enkele extra features:
    - compass_bearing/ course_over_ground
    - VgNr met tijdselement
 
Daarnaast zullen we de dataset resamplen met een frequentie van 3 minuten. Hiervoor is gekozen, omdat het gps-signaal frequenter wordt zodra de snelheid verhoogd. Dit zorgt ervoor dat de afstand tussen XY kleiner wordt, terwijl er harder is gevaren. 

In [ ]:
#Import packages
import matplotlib.pyplot as plt
plt.style.use('bmh')

import matplotlib.lines as mlines
from shapely.geometry import Polygon #Module for manipulation and analysis of geometric objects in the Cartesian plane.
import pandas as pd #This module provides high-performance, easy-to-use data structures and data analysis tools for Python
pd.options.mode.chained_assignment = None
from shapely.geometry import Point #The Point constructor takes positional coordinate values or point tuple parameters to create a single point.
import numpy as np
from geopy import distance
import csv
import time
import os
import datetime
import fnmatch
import concurrent.futures
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import mpld3
import folium
from geopy.distance import geodesic
import seaborn as sns

### Creëer sample uit een groot dataset
Om het model goed te kunnen begrijpen willen wij het model testen per schip. Gezien de ruwe dataset meerdere GB's groot is, runnen wij eenmalig per schip de code om het ruwe csv bestand in te lezen en het gekozen schip op te slaan in een nieuwe dataframe. Dit nieuwe dataframe slaan we op als csv, zodat we het nieuwe dataframe kunnen hergebruiken. 

Mocht het toch interessant zijn om niet per schip, maar naar het totale data bestand te kijken, kun je tot 10.000.000 rijen opslaan in een nieuwe datafram(ongeveer 2/3 dagen aan AIS-data). 

Uit de eerste 3 dagen van het bestand hebben wij 6000 verschillende schepen geselecteerd. Hieruit hebben wij een schip gekozen om te testen binnen ons model. 

In [ ]:
#Aan de hand van deze lijst met unique schepen in de dataset, konden wij boten selecteren om het model op te testen
# De lijst bevat een groupby die telt hoevaak het schip voorkomt in de eerste twee dagen van de dataset. 
# unique_ship_list = pd.read_csv("random_ship_list.csv")
# print(unique_ship_list)

In [ ]:
### HIERONDER DE METHODE OM DATA IN TE LEZEN. GEZIEN DEZE METHODE ERG LANG DUURT, KUN JE BETER 1x runnen en de output als CSV opslaan
### Naast het zoeken naar een specifiek schip, kun je ook de eerste 10.000.000 rijen opslaan als DF. Dit kun je doen door: if index == AANTAL RIJEN: Break;

# with open("DATAFILE.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
#     header = next(csvreader);
#     data = {}
#     for h in header:
#         data[h] = []
#     for index, row in enumerate(csvreader):
#          if row[5] == "MMSI_NUMMER":
#                 for h, v in zip(header, row):
#                     data[h].append(v);
                
# d = pd.DataFrame(data)
# d.to_csv("raw_MMSI_NUMMER.csv")
###---------------------------------------------------------------------------------------------------

In [ ]:
# # Navigatie status en snelheid bekijken van gekozen schip
# h = d.sort_values("t_speed")
# print(h.groupby(["t_speed"]).count())
# print(h.groupby(["t_navstatus"]).count())

### Opschonen data, voeg nieuwe kolommen met extra features toe. 

##### Inlezen csv bestand met 1 schip en selecteer juiste kolommen

In [ ]:
#Inlezen bestand
df = pd.read_csv("sample.csv",  usecols=["t_mmsi", "t_name", "t_updatetime", "t_longitude", "t_latitude", "t_speed", "t_orientation", "t_navstatus", "p_shiptype"] )

##### Omzetten UTC tijd en tijdseenheden in aparte kolommen 

In [ ]:
# Omzetten van UTC tijd naar locale tijd (Amsterdam) en verwijderen overbodige kolommen
df["Time"] = pd.DatetimeIndex(df['t_updatetime']).time
df['Date'] = pd.DatetimeIndex(df['t_updatetime']).date
df['DateTime_UTC'] = df.apply(lambda r : pd.datetime.combine(r['Date'],r['Time']),1)
df['DateTime_Local'] = df['DateTime_UTC'].dt.tz_localize('utc').dt.tz_convert('Europe/Amsterdam')
#Verwijder niet nodige tijdskolommen
del df["DateTime_UTC"]
del df["t_updatetime"]
del df['Time']
del df['Date']

# splitten tijdgegevens in kolommen
df['Date'] = pd.DatetimeIndex(df['DateTime_Local']).date
df['Time'] = pd.DatetimeIndex(df['DateTime_Local']).time
df['DateTime'] = df["DateTime_Local"]

## Hieronder nog meer code om tijd op te splitten
df['Year'] = pd.DatetimeIndex(df['DateTime']).year
df['Month'] = pd.DatetimeIndex(df['DateTime']).month
df['Day'] = pd.DatetimeIndex(df['DateTime']).day
df['Weeknr'] = pd.DatetimeIndex(df['DateTime']).week
df['Weekdag'] = pd.DatetimeIndex(df['DateTime']).weekday
df['Hour'] = pd.DatetimeIndex(df['DateTime']).hour
df["minute"] = pd.DatetimeIndex(df["DateTime"]).minute
df["sec"] = pd.DatetimeIndex(df["DateTime"]).second

#Verwijderen dubbele tijdskolommen 
del df['DateTime_Local']

##### Creëer volgnummer met tijdselement
In het volgende script gaan we data clusteren. Gezien "date-format" geen geschikt formaat is om mee te clusteren, berekenen we tijd om in seconden. Dit zorgt voor groot onleesbaar getal, dus kijken we naar het verschil en tellen we dit verschil bij elkaar op. De uitkomst hiervan noemen we het VgNr met tijdselement.   

In [ ]:
# Bereken datum in seconden
Year_sec = df["Year"] * 365 * 24 * 60 * 60
Month_sec = df["Month"] * 31 * 24 * 60 * 60
Day_sec = df["Day"] * 24 * 60 * 60
Hour_sec = df["Hour"] * 60 * 60
min_sec = df["minute"] * 60
sec_sec = df["sec"]

df["date_in_sec"] = Year_sec + Month_sec + Day_sec + Hour_sec + min_sec + sec_sec

# Creer volgnummer met tijdselement
# Bereken verschil tussen tijden en tel ze daarna bij elkaar op
df["VgNr"] = df["date_in_sec"].diff().cumsum()
# verwijder de NAN en zet float om in integers
df = df.dropna(subset = ["VgNr"]).reset_index(drop=True)
df["VgNr"] = df["VgNr"].astype(int)

##### Tel aantal berichten per minuut
Een andere feature die we kunnen maken met tijd is het aantal berichten per minuut. 

In [ ]:
# Maak kopie van dataframe vanwege latere merge
df_copy = df.copy()  
# Tel berichten per minuut door aantal rijen per minuut te tellen in df met een groupby. Merge daarna de resultaten met huidige df. 
count_messages_per_minute = df_copy.groupby(["Date", "Hour", "minute"])["t_mmsi"].count().reset_index(name="messages_per_minute")
df_copy_2 = pd.merge(df_copy, count_messages_per_minute, left_on=["Date", "Hour", "minute"], right_on=["Date", "Hour", "minute"])
# Je kunt ditzelfde doen per uur
count_messages_per_hour = df_copy_2.groupby(["Date", "Hour"])["t_mmsi"].count().reset_index(name="messages_per_hour")
df = pd.merge(df_copy_2, count_messages_per_hour, left_on=["Date", "Hour"], right_on=["Date", "Hour"])

##### Resample de data
Het AIS-signaal zendt frequenter data wanneer de snelheid van de boot versneld. Daardoor wordt de afstand tussen varende punten kleiner. Dit is niet goed voor het model, gezien we juist een grotere afstand willen tussen varende punten in vergelijking met stilliggende punten. Daarom samplen we de data naar 3 minuten. Er is gekozen voor 3 minuten, gezien dat de frequentie van het AIS-signaal is wanneer een schip stil ligt. 

Er zijn binnen Pandas twee manieren om te samplen: Grouper en Resample. Bij Grouper gooi je punten weg die binnen het geselecteerde tijdsinterval liggen, daarnaast behoudt je de originele tijd. Bij Resample verdeel je de data over gelijke tijdsintervallen (bijv 3min), dus behoud niet de orginele tijd. Punten die binnen de tijdsintervallen liggen, worden verwijderd. Je kunt o.a. per tijdinterval kiezen of je de sum, mean, eerste waarde of laatste waarde van de punten binnen het tijdsinterval pakt. Daarnaast voegt hij puntjes toe, wanneer de tijdsinterval langer is dan aangegeven. Deze punten krijgen een tijdindex, maar de rest van de kolommen krijgen de value NaN. Het is mogelijk om deze NaN te vullen met bijv de waarde die voor of na de NaN te vinden is. 

Binnen deze opdracht hebben we gekozen voor de functie Resample. Op stoplocaties kan het zijn dat de AIS heeft uitgestaan waardoor het verschil in tijd tussen twee punten kan oplopen tot meerdere uren. Door hier meer punten toe te voegen (met alle dezelfde waarde als het punt daarvoor of daarna), versterk je het stop signaal. Dit is positief voor het resultaat. 

Let op: wanneer het signaal tijdens het varen wegvalt, zullen hier ook extra punten worden toevoegd. Controleer of dit tot problemen leidt (voor de schepen die wij hebben getest, leidde dit niet tot problemen). 

In [ ]:
# nieuwe df gezien latere merge 
df_sample = df.copy()

# Gebruik Resample om gelijke tijdsintervallen te creëren van 3 minuten. Gebruik max() om de laatste rij te selecteren voor de tijdsintval (je kunt hier ook bijv. sum, mean of min() gebruiken)
df_sample = df_sample.set_index("DateTime").resample('3T').max()
# Gebruik fillna om een waarde toe te kennen aan de punten die zijn toegevoegd (deze gegevens hebben een NA gekregen). De backfill methode gebuikt de volgende valide waarde uit de dataset om de gab te vullen. Je kunt ook ffill gebruiken , deze gebruik juist de laatste valide waarde uit de dataset om de gab te dichten. 
df_sample = df_sample.fillna(method="backfill")

# Andere methode om te samplen is via Grouper. Deze methode behoudt de origele tijd, waardoor de tijdsintervallen niet allemaal gelijk zijn (maar wel een eerlijker beeld geven). Volgens mij is het niet mogelijk om te upsamplen via grouper (kan hier geen documentatie over vinden). Vandaar niet interessant voor ons project. 
# df_sample = df_sample.groupby(['t_name', pd.Grouper(key='DateTime_index', freq='3Min')]).max()

In [ ]:
# Met de resample tool worden de string-kolommen verwijderd, met merge weer toevoegen

# Ivm de merge ontstaan er dubbele kolommen, deze kun je verwijderen met de onderstaande functies
def drop_y(df):
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    
def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)
            
# df_merge = pd.merge(df_sample, df, left_index=True, how="inner", on = "VgNr")
df_merge = pd.merge(df_sample, df, how="left", on = "VgNr")
drop_y(df=df_merge)
rename_x(df=df_merge)

df = df_merge.copy()
# print(df)


##### Naast VgNr met tijdselement ook ID meegeven
We willen testen hoe het clustermodel omgaat met tijd (VgNr) en hoe deze omgaat met volgorde van punten zonder tijdselement (ID). Dit doen wij omdat er soms veel tijd zit tussen de punten wanneer een schip stil ligt. Dit is nadelig voor het model.

In [ ]:
# Geef elke rij een id door index naar een kolom om te zetten
df = df.reset_index()
df["ID"] = df.index.tolist()

#### Creëer features met koers
Het is mogelijk om tussen twee punten de hoek te berekenen en dit om te zetten in "course over ground"/"compass bearing".

In [ ]:
## 1/3 feature koers: eerste afgeleide van t_orientation tussen twee punten

## bereken eerste afgeleide + zet om in absolute
df["t_orientation_dff"] = df["t_orientation"].diff(1).abs()
## Gezien 0 / 360 allebei het noorden zijn trekken we 360 graden van het totaal bij punten boven de 180 graden, zodat met eenheden onder de 180 graden
df.loc[df["t_orientation_dff"] > 180, "t_orientation_dff"] = (df.loc[df["t_orientation_dff"] > 180, "t_orientation_dff"] - 360).abs()

In [ ]:
## 2/3 feature koers: bereken hoek en graden tussen twee punten

# verschil lat1 en lat2
x1 = df.t_latitude
x2 = df.t_latitude.shift(-1) 
deltax = x2 -x1

# verschil lon1 en lon2
y1 = df.t_longitude
y2 = df.t_longitude.shift(-1)
deltay = y2 - y1

# bereken hoek 
df["angle_rad"] = np.arctan2(deltay,deltax).abs()

# bereken graden
df["angle_deg"] = df["angle_rad"]*180.0/np.pi

# Bereken eerste afgeleide + zet om in absolute
df["angle_deg_diff"] = df["angle_deg"].diff().abs()
## Gezien 0 / 360 allebei het noorden zijn trekken we 360 graden van het totaal bij punten boven de 180 graden, zodat met eenheden onder de 180 graden
df.loc[df["angle_deg_diff"] > 180, "angle_deg_diff"] = (df.loc[df["angle_deg_diff"] > 180, "angle_deg_diff"] - 360).abs()

In [ ]:
## 3/3 feature koers: ingewikkelder formule om compass bearing uit te rekenen
## Ik snap niet helemaal wat hij doet

# De eerste functie is een wat uit
def calculate_initial_compass_bearing(df):
#     Calculates the bearing between two points.
#     The formulae used is the following:
#         θ = atan2(sin(Δlong).cos(lat2),
#                   cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
#     :Parameters:
#       - `pointA: The tuple representing the latitude/longitude for the
#         first point. Latitude and longitude must be in decimal degrees
#       - `pointB: The tuple representing the latitude/longitude for the
#         second point. Latitude and longitude must be in decimal degrees
#     :Returns:
#       The bearing in degrees
#     :Returns Type:
#       float

#     if (type(pointA) != tuple) or (type(pointB) != tuple):
#         raise TypeError("Only tuples are supported as arguments")
#     lat = df[]
    lat = df["t_latitude"].iloc[:-1]
    lat2 = df["t_latitude"].shift(-1).dropna()
    lat = np.asarray(lat)
    lat2 = np.asarray(lat2)
    lon = df["t_longitude"].iloc[:-1]
    lon2 = df["t_longitude"].shift(-1).dropna()
    lon = np.asarray(lon)
    lon2 = np.asarray(lon2)

    diffLong = np.radians(lon2 - lon)

    x = np.sin(diffLong) * np.cos(lat2)
    y = np.cos(lat) * np.sin(lat2) - (np.sin(lat)
            * np.cos(lat2) * np.cos(diffLong))

    initial_bearing = np.arctan2(x, y)

#     # Now we have the initial bearing but math.atan2 return values
#     # from -180° to + 180° which is not what we want for a compass bearing
#     # The solution is to normalize the initial bearing as shown below
    initial_bearing = np.degrees(initial_bearing)
    df = df.iloc[:-1]
    df["compass_bearing"] = (initial_bearing + 360) % 360
    
    # Bereken eerste afgeleide + zet om in absolute
    df["compass_dff"] = df["compass_bearing"].diff().abs()
    ## Gezien 0 / 360 allebei het noorden zijn trekken we 360 graden van het totaal bij punten boven de 180 graden, zodat met eenheden onder de 180 graden
    df.loc[df["compass_dff"] > 180, "compass_dff"] = (df.loc[df["compass_dff"] > 180, "compass_dff"] - 360).abs()

    return(df)

df = calculate_initial_compass_bearing(df=df)

##### Sla csv op

In [ ]:
## NAN door verschil berekenen koers
df = df.dropna(subset = ["t_orientation_dff"]).reset_index(drop=True)
## Wellicht zitten NAN's in de t_latitude/ t_longitude/ t_navstatus
df = df.dropna(subset = ["t_latitude", "t_longitude"]).reset_index(drop=True)
df = df.dropna(subset = ["t_navstatus"]).reset_index(drop=True)
df["t_navstatus"] = df["t_navstatus"].astype(int)
## Maak csv van preprocessing
df.to_csv("sample_AIS_preprocessing.csv")